# APIs Langchain

We we'll be doing some test evaluating:

- Asking general questions to some API.


In [6]:
from langchain.chains import APIChain
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.prompts.prompt import PromptTemplate

from langchain.llms import GPT4All


Loading the LLM. In this case we are loading GPT4All


In [7]:
from tqdm import tqdm
from pathlib import Path
import requests
local_path = (
    # replace with your desired local file path
    "./models/ggml-gpt4all-l13b-snoozy.bin"
)


Path(local_path).parent.mkdir(parents=True, exist_ok=True)

# Example model. Check https://github.com/nomic-ai/gpt4all for the latest models.
url = 'http://gpt4all.io/models/ggml-gpt4all-l13b-snoozy.bin'

# send a GET request to the URL to download the file. Stream since it's large
response = requests.get(url, stream=True)

# open the file in binary mode and write the contents of the response to it in chunks
# This is a large file, so be prepared to wait.
with open(local_path, 'wb') as f:
    for chunk in tqdm(response.iter_content(chunk_size=8192)):
        if chunk:
            f.write(chunk)


993259it [22:39, 730.76it/s] 


Now the GPT4All model is loaded into the `llm` variable.


In [9]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

# If you want to use a custom model add the backend parameter
# Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
llm = GPT4All(model=local_path, backend="gptj",
              callbacks=callbacks, verbose=True)


Found model file at  ./models/ggml-gpt4all-l13b-snoozy.bin


objc[71874]: Class GGMLMetalClass is implemented in both /Users/ernesto/work/github/MosAPIc/.venv/lib/python3.11/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libreplit-mainline-metal.dylib (0x2b180c208) and /Users/ernesto/work/github/MosAPIc/.venv/lib/python3.11/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libllamamodel-mainline-metal.dylib (0x2b1a58208). One of the two will be used. Which one is undefined.
llama.cpp: loading model from ./models/ggml-gpt4all-l13b-snoozy.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: n_parts    = 1


Found model file at  ./models/ggml-gpt4all-l13b-snoozy.bin


........................................................................
llama_init_from_file: kv self size  = 1600.00 MB


## Interacting with the weather example


In [10]:
from langchain.chains.api import open_meteo_docs

chain_new = APIChain.from_llm_and_api_docs(
    llm, open_meteo_docs.OPEN_METEO_DOCS, verbose=True)


In [14]:
# chain_new.run('What is the weather like now in Costa Rica in celsius?')
chain_new.run(
    'What is the weather like right now in Uruguay in degrees Fahrenheit?')




> Entering new  chain...
 https://api.open-meteo.com/v1/forecast?latitude=34.9025&longitude=-56.7875&hourly=[temperature_2m,snowfall,rain,showers,weathercode]&daily=[precipitation_amount,windspeed,winddirection,humidity,dewpoint,visibility,sunrise,sunset,solarnoon,solartime,moonset,moontime,phaseofthemoon,magneticfieldstrength,airpressure]&current_weather=true
timeformat=iso8601
timezone=GMT+0 https://api.open-meteo.com/v1/forecast?latitude=34.9025&longitude=-56.7875&hourly=[temperature_2m,snowfall,rain,showers,weathercode]&daily=[precipitation_amount,windspeed,winddirection,humidity,dewpoint,visibility,sunrise,sunset,solarnoon,solartime,moonset,moontime,phaseofthemoon,magneticfieldstrength,airpressure]&current_weather=true
timeformat=iso8601
timezone=GMT+0
{"error":true,"reason":"Value of type 'Bool' required for key 'current_weather'."}
 The error message indicates that a Boolean value was expected for the "current\_weather" parameter, but it received an invalid input (most likely 

' The error message indicates that a Boolean value was expected for the "current\\_weather" parameter, but it received an invalid input (most likely due to missing or incorrect data). Therefore, the API call did not return any useful information about the current weather in Uruguay at this time.'